In [1]:
import os
from pathlib import Path
import re
from tqdm import tqdm
from PIL import Image

In [2]:
def get_accuracy(mei, sixteenths):
    dur_to_sixteenth = {'1"': 16, '2"': 8, '4"': 4, '8"': 2, '16': 1, '32': 0.5, '64':0.25, 'breve': 32, 'long': 64}
    bar_num = 0
    bar_end = False
    accurate = 0
    inaccurate = 0
    metercount, meterunit = None, None
    num_sixteenths = 0
    last_duration = 0
    current_dots = 0
    ignored = 0
    ignoring = False
    multirest = False
    stop = False
    for word in mei:
        if word[:6] == '<layer':
            num_sixteenths = 0
            last_duration = 0
            bar_num += 1
            ignored = 0
            ignoring = False
            current_dots = 0
            multirest = False
        elif word == "</layer>":   # it only updates accuracy if last measure is complete?
            if bar_num != 1 and not ignoring:  # if it wasn't the first bar, record accurate or inaccurate
                if num_sixteenths == sixteenths:
                    accurate += 1
                elif multirest:
                    accurate += 1
                else:
                    inaccurate += 1 
            num_sixteenths = 0
            last_duration = 0
            ignored = 0
            ignoring = False
            current_dots = 0
            multirest = False
        elif word[:5] == 'dur="':
            if word[5:7] in dur_to_sixteenth.keys():
                dur = word[5:7]
            elif word[5:10] == 'breve':
                dur = word[5:10]
            elif word[5:9] == 'long':
                dur = word[5:9]
            else:
                print(word)
#                 print(file)# invalid (shouldn't be included)
                
            if not ignoring and (not stop):
                if current_dots == 1:
                    last_duration = dur_to_sixteenth[dur] + (dur_to_sixteenth[dur] * 0.5)
                    current_dots = 0
                elif current_dots == 2:
                    last_duration = dur_to_sixteenth[dur] + (dur_to_sixteenth[dur] * 0.75)
                    current_dots = 0
                else:
                    last_duration = dur_to_sixteenth[dur]
                num_sixteenths += last_duration
        elif word[:6] == 'dots="':
            if last_duration != 0:
                if word[6] == '1':
                    current_dots += 1
                elif word[6] == '2':
                    current_dots += 2
            else:
                ignored += 1
                ignoring = True
        elif word == '<multiRest':
            multirest = True
            last_duration = 0
        elif (bar_num != 0) and word[:5] == 'meter':  # stop if new time signature
            stop = True
            last_duration = 0
        elif word[:5] == 'grace':
            # remove last duration
            num_sixteenths -= last_duration
            last_duration = 0
        
    return accurate, inaccurate, ignored
            

# GPT-2

In [90]:
# might need to change code, but basically
    # make a list of filenames that are invalid

failed_engravings = Path('/home/ibukey/ttmp/failed_engravings_marcos.txt')
failed_list = []
with open(failed_engravings) as f:
    t = f.read().split()
    for file in t:
        failed_list.append(file[48:-4])
len(failed_list)

664

In [91]:
# might need to change code, but basically
    # make lists of files that end in mei and are not in the failed list of files

MEI_2_4 = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/2-4') if (file[-4:] == '.mei' and '2-4/'+file[:-4] not in failed_list)]
MEI_3_4 = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/3-4') if (file[-4:] == '.mei' and '3-4/'+file[:-4] not in failed_list)]
MEI_C = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/C') if (file[-4:] == '.mei' and 'C/'+file[:-4] not in failed_list)]
MEI_5_4 = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/5-4') if (file[-4:] == '.mei' and '5-4/'+file[:-4] not in failed_list)]
MEI_6_4 = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/6-4') if (file[-4:] == '.mei' and '6-4/'+file[:-4] not in failed_list)]
MEI_7_4 = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/7-4') if (file[-4:] == '.mei' and '7-4/'+file[:-4] not in failed_list)]
MEI_3_8 = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/3-8') if (file[-4:] == '.mei' and '3-8/'+file[:-4] not in failed_list)]
MEI_6_8 = [file for file in os.listdir('/home/macosta/ttmp/generated-mei-timesigs-final/6-8') if (file[-4:] == '.mei' and '6-8/'+file[:-4] not in failed_list)]

In [92]:
# check the total num files generated (ending in mei) and total num files that are valid
# check that the difference is equal to the num files in failed_engravings
PATH = '/home/macosta/ttmp/generated-mei-timesigs-final/'
total_count = 0
invalid_count = len(failed_list)
valid_count = len(MEI_2_4) + len(MEI_3_4) + len(MEI_C) + len(MEI_5_4) + len(MEI_6_4) + \
            len(MEI_7_4) + len(MEI_3_8) + len(MEI_6_8)
# total mei
for folder in os.listdir(PATH):
    for file in os.listdir(PATH + folder):
        if file[-4:] == '.mei':
            total_count += 1

In [93]:
(total_count - invalid_count == valid_count), total_count, invalid_count, valid_count   

(True, 16000, 664, 15336)

In [94]:
results_2_4 = []
for i in range(len(MEI_2_4)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/2-4/' + MEI_2_4[i]) as f:
        t = f.read().split()
        results_2_4.append(get_accuracy(t, 2*4))

In [95]:
acc_2_4 = [sum(x) for x in zip(*results_2_4)]
print(acc_2_4)
acc_2_4[0] / (acc_2_4[0] + acc_2_4[1])

[5681, 969, 0]


0.8542857142857143

In [96]:
results_3_4 = []
for i in range(len(MEI_3_4)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/3-4/' + MEI_3_4[i]) as f:
        t = f.read().split()
        results_3_4.append(get_accuracy(t, 3*4))

In [97]:
acc_3_4 = [sum(x) for x in zip(*results_3_4)]
print(acc_3_4)
acc_3_4[0] / (acc_3_4[0] + acc_3_4[1])

[5302, 796, 0]


0.8694653984913087

In [98]:
results_C = []
for i in range(len(MEI_C)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/C/' + MEI_C[i]) as f:
        t = f.read().split()
        results_C.append(get_accuracy(t, 4*4))

In [99]:
acc_C = [sum(x) for x in zip(*results_C)]
print(acc_C)
acc_C[0] / (acc_C[0] + acc_C[1])

[3743, 1067, 0]


0.7781704781704781

In [100]:
results_5_4 = []
for i in range(len(MEI_5_4)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/5-4/' + MEI_5_4[i]) as f:
        t = f.read().split()
        results_5_4.append(get_accuracy(t, 5*4))

In [101]:
acc_5_4 = [sum(x) for x in zip(*results_5_4)]
print(acc_5_4)
acc_5_4[0] / (acc_5_4[0] + acc_5_4[1])

[396, 3995, 0]


0.09018446823047142

In [102]:
results_6_4 = []
for i in range(len(MEI_6_4)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/6-4/' + MEI_6_4[i]) as f:
        t = f.read().split()
        results_6_4.append(get_accuracy(t, 6*4))

In [103]:
acc_6_4 = [sum(x) for x in zip(*results_6_4)]
print(acc_6_4)
acc_6_4[0] / (acc_6_4[0] + acc_6_4[1])

[2093, 1426, 0]


0.5947712418300654

In [104]:
results_7_4 = []
for i in range(len(MEI_7_4)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/7-4/' + MEI_7_4[i]) as f:
        t = f.read().split()
        results_7_4.append(get_accuracy(t, 7*4))

dur="g"/>


In [105]:
acc_7_4 = [sum(x) for x in zip(*results_7_4)]
print(acc_7_4)
acc_7_4[0] / (acc_7_4[0] + acc_7_4[1])

[304, 3362, 1]


0.08292416803055101

In [106]:
results_3_8 = []
for i in range(len(MEI_3_8)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/3-8/' + MEI_3_8[i]) as f:
        t = f.read().split()
        results_3_8.append(get_accuracy(t, 3*2))

In [107]:
acc_3_8 = [sum(x) for x in zip(*results_3_8)]
print(acc_3_8)
acc_3_8[0] / (acc_3_8[0] + acc_3_8[1])

[6463, 597, 0]


0.9154390934844192

In [108]:
results_6_8 = []
for i in range(len(MEI_6_8)):
    with open('/home/macosta/ttmp/generated-mei-timesigs-final/6-8/' + MEI_6_8[i]) as f:
        t = f.read().split()
        results_6_8.append(get_accuracy(t, 6*2))

In [109]:
acc_6_8 = [sum(x) for x in zip(*results_6_8)]
print(acc_6_8)
acc_6_8[0] / (acc_6_8[0] + acc_6_8[1])

[4191, 857, 0]


0.830229793977813

In [110]:
overall = [sum(x) for x in zip(acc_2_4,acc_3_4,acc_C,acc_5_4,acc_6_4,acc_7_4,acc_3_8,acc_6_8)]
print(overall)
overall[0] / (overall[0] + overall[1])

[28173, 13069, 1]


0.6831143009553368

# AWD-LSTM

In [16]:
# same as above

failed_engravings = Path('/home/ibukey/ttmp/failed_engravings_irmak.txt')
failed_list = []
with open(failed_engravings) as f:
    t = f.read().split()
    for file in t:
        failed_list.append(file[33:-4])  # might need to change
len(failed_list)

129

In [17]:
# same as above
MEI_2_4 = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:9] == 'pred_2_4_' and file[:-4] not in failed_list)]
MEI_3_4 = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:9] == 'pred_3_4_' and file[:-4] not in failed_list)]
MEI_C = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:12] == 'pred_common_' and file[:-4] not in failed_list)]
MEI_5_4 = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:9] == 'pred_5_4_' and file[:-4] not in failed_list)]
MEI_6_4 = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:9] == 'pred_6_4_' and file[:-4] not in failed_list)]
MEI_7_4 = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:9] == 'pred_7_4_' and file[:-4] not in failed_list)]
MEI_3_8 = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:9] == 'pred_3_8_' and file[:-4] not in failed_list)]
MEI_6_8 = [file for file in os.listdir('/home/ibukey/ttmp/mei-uncleaned') if (file[:9] == 'pred_6_8_' and file[:-4] not in failed_list)]

In [19]:
# check the total num files generated (ending in mei) and total num files that are valid
# check that the difference is equal to the num files in failed_engravings
PATH = '/home/ibukey/ttmp/mei-uncleaned/'
total_count = 0
invalid_count = len(failed_list)
valid_count = len(MEI_2_4) + len(MEI_3_4) + len(MEI_C) + len(MEI_5_4) + len(MEI_6_4) + \
            len(MEI_7_4) + len(MEI_3_8) + len(MEI_6_8)
# total mei

for file in os.listdir(PATH):
    if file[-4:] == '.mei':
        total_count += 1

In [20]:
(total_count - invalid_count == valid_count), total_count, invalid_count, valid_count   

(True, 15029, 129, 14900)

In [21]:
results_2_4 = []
for file in MEI_2_4:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_2_4.append(get_accuracy(t, 2*4))

In [22]:
acc_2_4 = [sum(x) for x in zip(*results_2_4)]
print(acc_2_4)
acc_2_4[0] / (acc_2_4[0] + acc_2_4[1])

[1675, 3079, 0]


0.352334875893984

In [25]:
results_3_4 = []
for file in MEI_3_4:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_3_4.append(get_accuracy(t, 3*4))

In [26]:
acc_3_4 = [sum(x) for x in zip(*results_3_4)]
print(acc_3_4)
acc_3_4[0] / (acc_3_4[0] + acc_3_4[1])

[3469, 1529, 0]


0.6940776310524209

In [27]:
results_C = []
for file in MEI_C:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_C.append(get_accuracy(t, 4*4))

In [28]:
acc_C = [sum(x) for x in zip(*results_C)]
print(acc_C)
acc_C[0] / (acc_C[0] + acc_C[1])

[3274, 1144, 0]


0.7410593028519692

In [29]:
results_5_4 = []
for file in MEI_5_4:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_5_4.append(get_accuracy(t, 5*4))

In [30]:
acc_5_4 = [sum(x) for x in zip(*results_5_4)]
print(acc_5_4)
acc_5_4[0] / (acc_5_4[0] + acc_5_4[1])

[173, 5227, 0]


0.03203703703703704

In [31]:
results_6_4 = []
for file in MEI_6_4:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_6_4.append(get_accuracy(t, 6*4))

In [32]:
acc_6_4 = [sum(x) for x in zip(*results_6_4)]
print(acc_6_4)
acc_6_4[0] / (acc_6_4[0] + acc_6_4[1])

[576, 3722, 0]


0.13401582131223824

In [33]:
results_7_4 = []
for file in MEI_7_4:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_7_4.append(get_accuracy(t, 7*4))

In [34]:
acc_7_4 = [sum(x) for x in zip(*results_7_4)]
print(acc_7_4)
acc_7_4[0] / (acc_7_4[0] + acc_7_4[1])

[132, 5454, 0]


0.023630504833512353

In [40]:
results_3_8 = []
for file in MEI_3_8:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_3_8.append(get_accuracy(t, 3*2))

In [41]:
acc_3_8 = [sum(x) for x in zip(*results_3_8)]
print(acc_3_8)
acc_3_8[0] / (acc_3_8[0] + acc_3_8[1])

[929, 3191, 0]


0.22548543689320388

In [42]:
results_6_8 = []
for file in MEI_6_8:
    with open('/home/ibukey/ttmp/mei-uncleaned/' + file) as f:
        t = f.read().split()
        results_6_8.append(get_accuracy(t, 6*2))

In [43]:
acc_6_8 = [sum(x) for x in zip(*results_6_8)]
print(acc_6_8)
acc_6_8[0] / (acc_6_8[0] + acc_6_8[1])

[3532, 822, 0]


0.8112080845199816

In [39]:
overall = [sum(x) for x in zip(acc_2_4,acc_3_4,acc_C,acc_5_4,acc_6_4,acc_7_4,acc_3_8,acc_6_8)]
print(overall)
overall[0] / (overall[0] + overall[1])

[13760, 24168, 0]


0.3627926597764185